In [20]:
from iroha import IrohaCrypto
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import json
from iroha_helper import *
from new_helper import *
from super_helper import *
from ipfs_functions import *
from loguru import logger

In [21]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 1

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


In [22]:
# Customize the logger format
logger.remove()
logger.add(
    sink=lambda msg: print(msg, end=""),
    format="<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | "
           "<level>{level: <8}</level> | "
           "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - "
           "{message}",
    colorize=True,  # Enable colors for supported terminals
)

4

In [23]:
#Manually resets the index on execution
# recreate_index() 

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [24]:
hash = create_detail_contract()
integration_helpers.get_engine_receipts_result(hash)

2025-01-13 23:08:54.791 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-13 23:08:54.809 | INFO     | iroha_helper:create_detail_contract:55 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-13 23:08:54.814 | INFO     | iroha_helper:create_detail_contract:55 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-13 23:08:56.953 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-13 23:08:56.983 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-13 23:08:56.983 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-13 23:08:56.983 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-13 23:08:57.000 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-13 23:08:57.001 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [25]:
# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-01-13 23:08:57.024 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-13 23:08:57.044 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-13 23:08:57.044 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-13 23:08:57.058 | INFO     | iroha_helper:set_account_detail:296 - None
2025-01-13 23:08:57.059 | INFO     | iroha_helper:set_account_detail:298 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-13 23:08:57.060 | INFO     | iroha_helper:set_account_detail:298 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-13 23:09:00.005 | INFO     | iroha_helper:set_account_detail:298 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-13 23:09:00.046 | INFO     | iroha_helper:set_account_detail:298 - ('COMMITTED', 5, 0)
2025-01-13 23:09:00.047 | INFO     | iroha_helper:tracer:246 - 	Leaving "set_account_detail"
2025-01-13 23:09:00.047 | INFO     | iroha_helper:tracer:244 - 	Entering "set_acc

3 - Queries Iroha 1 for User account and checks its values

In [26]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# logger.info(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# logger.info(response)

user_data = response.account_detail_response
user_details = user_data.detail

logger.info(f'User Account id = {user_account}, {user_details}')

2025-01-13 23:09:03.136 | INFO     | __main__:<module>:11 - User Account id = {'account_id': 'romantic_johnson@test'}, { "admin@test" : { "linked_project" : "77323@test", "user_json_ld_cid" : "QmdTLM5zBQe4CivjMxW5wXJMzErPo3qWXYvXgHHG3yTqbe" } }


6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [27]:
# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
logger.info(user_details_dict)

# Now you can access the specific key like this
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(project_id)

2025-01-13 23:09:03.193 | INFO     | __main__:<module>:3 - {'admin@test': {'linked_project': '77323@test', 'user_json_ld_cid': 'QmdTLM5zBQe4CivjMxW5wXJMzErPo3qWXYvXgHHG3yTqbe'}}
2025-01-13 23:09:03.194 | INFO     | __main__:<module>:7 - 77323@test


In [28]:
schema = get_schema() #super_helper.py

logger.info(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

2025-01-13 23:09:03.252 | INFO     | __main__:<module>:3 - <Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>


2025-01-13 23:09:03.420 | INFO     | iroha_helper:tracer:244 - 	Entering "create_contract"
2025-01-13 23:09:03.425 | INFO     | iroha_helper:create_contract:263 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-13 23:09:03.426 | INFO     | iroha_helper:create_contract:263 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-13 23:09:06.117 | INFO     | iroha_helper:create_contract:263 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-13 23:09:06.145 | INFO     | iroha_helper:create_contract:263 - ('COMMITTED', 5, 0)
2025-01-13 23:09:06.146 | INFO     | iroha_helper:tracer:246 - 	Leaving "create_contract"
2025-01-13 23:09:06.146 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-13 23:09:06.164 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-13 23:09:06.164 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-13 23:09:06.178 | INFO     | iroha_helper:set_account_detail:296 - None
2025-

In [29]:
account_detail = get_account_detail(project_id)
logger.info(f"{project_id}, {account_detail}")


2025-01-13 23:09:13.010 | INFO     | __main__:<module>:2 - 77323@test, { "admin@test" : { "file_1" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Qme5cwgyMn5R2LRqT6jvrg4GfjuauPKwGR4m6njHpzhAtR", "file_2" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmWa6PngcSfzdn677h7j8fqQyk7xFp2PVpJzj8whpJUUw9", "linked_user" : "romantic_johnson@test", "project_metadata_cid" : "QmaWfoLPTka6WB8qTFiJ7YhZs2k3XPkdDDocEmTS5L7ZF7" } }


In [30]:
# Perform a keyword search
keyword = "paper"
search_results, project_ids_with_cids = search_index(index, keyword)

2025-01-13 23:09:13.067 | INFO     | super_helper:search_index:230 - Starting keyword search...
2025-01-13 23:09:13.067 | INFO     | super_helper:search_index:231 - Keyword: 'paper'
2025-01-13 23:09:13.077 | INFO     | super_helper:search_index:242 - Search successful: Found 16 result(s).
2025-01-13 23:09:13.079 | INFO     | super_helper:search_index:244 - 1. Project Id: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, Metadata CID: QmWa6PngcSfzdn677h7j8fqQyk7xFp2PVpJzj8whpJUUw9, Title: deep learning in insurance: accuracy and model interpretability using tabnet
2025-01-13 23:09:13.081 | INFO     | super_helper:search_index:244 - 2. Project Id: 37355@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmQvGdmxoDtMFN3ciARqKANbJ6EGsXowfncs59LKb356s4, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-13 23:09:13.083 | INFO     | super_helper:search_index:244 - 3. Project I

In [31]:
# Check if there are no search results
if not search_results:
    logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
else:
    # Process each dictionary in search results
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        if not project_id or not file_cid or not metadata_cid:
            logger.error(f"Missing required data in result: {result_dict}")
            continue

        # Log the retrieved project details
        logger.info(f"Processing Project ID: {project_id}")
        logger.info(f"File CID: {file_cid}")
        logger.info(f"Metadata CID: {metadata_cid}")
        # file_metadata_json = download_json_from_ipfs(metadata_cid)
        # logger.info("file_metadata_json:", file_metadata_json)
        

        # Fetch project details from the blockchain
        project_details = get_account_detail(project_id)
        if not project_details:
            logger.error(f"No project details found for Project ID: {project_id}.")
            continue

        logger.info(f"Fetched project details for {project_id}: {project_details}")

        # Parse blockchain data
        try:
            blockchain_data = json.loads(project_details)
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding project details JSON for {project_id}: {e}")
            continue

        # Validate file CID and fetch project details
        validation_result = fetch_project_details(file_cid, blockchain_data)
        logger.info(f"Valid Result for {project_id} is {validation_result}.")
        if validation_result["is_valid"]:
            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")
            file_metadata_cid = validation_result.get("metadata_cid")
            
            # download_file(file_metadata_json, download_path, project_id, file_cid)

            logger.info(f"Valid File CID for {project_id}.")
            logger.info(f"Project Metadata CID: {project_metadata_cid}")
            logger.info(f"Linked User: {linked_user}")

            # Fetch and process metadata and user details
            if project_metadata_cid:
                logger.info(f"Processing project metadata CID: {project_metadata_cid}")
                project_metadata = download_json_from_ipfs(project_metadata_cid)
                logger.info(f"Downloaded project metadata: {project_metadata}")

            if linked_user:
                logger.info(f"Processing linked user: {linked_user}")
                user_details = get_account_detail(linked_user)
                try:
                    user_details = json.loads(user_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                    continue

                user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid", None)
                if user_json_ld_cid:
                    logger.info(f"User JSON-LD CID: {user_json_ld_cid}")
                    user_metadata = download_json_from_ipfs(user_json_ld_cid)
                    logger.info(f"Downloaded user metadata: {user_metadata}")
                else:
                    logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")
            
            if metadata_cid:
                logger.info(f"Processing metadata CID: {metadata_cid}")
                file_metadata = download_json_from_ipfs(metadata_cid)
                file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                logger.info(f"Downloaded file metadata: {metadata_cid}")
                logger.info(f"file metadata: {file_metadata}")
                logger.info(f"file metadata JSON: {file_metadata_json}")
            
        
        else:
            logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")

2025-01-13 23:09:13.189 | INFO     | __main__:<module>:16 - Processing Project ID: 77323@test
2025-01-13 23:09:13.190 | INFO     | __main__:<module>:17 - File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-13 23:09:13.190 | INFO     | __main__:<module>:18 - Metadata CID: QmWa6PngcSfzdn677h7j8fqQyk7xFp2PVpJzj8whpJUUw9
2025-01-13 23:09:13.207 | INFO     | __main__:<module>:29 - Fetched project details for 77323@test: { "admin@test" : { "file_1" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Qme5cwgyMn5R2LRqT6jvrg4GfjuauPKwGR4m6njHpzhAtR", "file_2" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmWa6PngcSfzdn677h7j8fqQyk7xFp2PVpJzj8whpJUUw9", "linked_user" : "romantic_johnson@test", "project_metadata_cid" : "QmaWfoLPTka6WB8qTFiJ7YhZs2k3XPkdDDocEmTS5L7ZF7" } }
2025-01-13 23:09:13.208 | INFO     | super_helper:fetch_project_details:288 - Validating and fetching details for CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-13 23:09:13.208 | INFO     | super_hel

2025-01-13 23:09:13.299 | INFO     | __main__:<module>:79 - Downloaded file metadata: QmWa6PngcSfzdn677h7j8fqQyk7xFp2PVpJzj8whpJUUw9
2025-01-13 23:09:13.299 | INFO     | __main__:<module>:80 - file metadata: {'Content-Length': '1697865', 'Content-Type': 'application/pdf', 'CreationDate--Text': '7th February 2023', 'CrossMarkDomains[1]': 'elsevier.com', 'CrossMarkDomains[2]': 'sciencedirect.com', 'CrossmarkDomainExclusive': 'true', 'CrossmarkMajorVersionDate': '2010-04-23', 'ElsevierWebPDFSpecifications': '7.0', 'X-TIKA:Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'X-TIKA:embedded_depth': '0', 'X-TIKA:parse_time_millis': '62', 'access_permission:assemble_document': 'true', 'access_permission:can_modify': 'true', 'access_permission:can_print': 'true', 'access_permission:can_print_d

With Reusable Helper Function for Block Separation and Tracing

A helper function, with_logging_block, which accepts a block name, the code block to execute, and optional parameters for context-specific tracing.

Cleanly separates each logical block, improving readability, modularity, and error tracing.

Includes the output of the search results explicitly logged within the Keyword Search block. This will ensure all search results are printed for visibility and debugging purposes.

In [32]:
from contextlib import contextmanager
import json
import sys


@contextmanager
def with_logging_block(block_name, logger):
    """
    A reusable context manager for logging structured execution blocks.

    Args:
        block_name (str): The name of the block being executed.
        logger (Logger): The logger instance.
    """
    try:
        logger.info("\n" + "=" * 50)
        logger.info(f"STARTING BLOCK: {block_name}")
        logger.info("=" * 50)
        yield  # Code within the `with` block will execute here
    except Exception as e:
        logger.error(f"An error occurred in block '{block_name}': {e}. Exiting.")
        sys.exit(1)  # Graceful exit on error
    finally:
        logger.info(f"COMPLETED BLOCK: {block_name}")
        logger.info("-" * 50 + "\n")


# Main Code Logic

# with with_logging_block("Keyword Search", logger):
#     if not search_results:
#         logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
#         sys.exit(1)
#     logger.info(f"Search results for keyword '{keyword}':")
#     for idx, result in enumerate(search_results, 1):
#         logger.info(f"Result {idx}: {json.dumps(result, indent=2)}")

with with_logging_block("Keyword Search", logger):
    if not search_results:
        logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
        sys.exit(1)
    
    # Log the total number of search results
    total_results = len(search_results)
    logger.info(f"Total search results found for keyword '{keyword}': {total_results}")
    
    # List the `project_id` and `file_cid` for each search result
    logger.info("Listing all search results:")
    for idx, result in enumerate(search_results, 1):
        project_id = result.get('project_id', 'N/A')
        file_cid = result.get('file_cid', 'N/A')
        logger.info(f"Result {idx}: Project ID: {project_id}, File CID: {file_cid}")
    
    # Log the full details of search results
    logger.info(f"Full search results for keyword '{keyword}':")
    for idx, result in enumerate(search_results, 1):
        logger.info(f"Result {idx}: {json.dumps(result, indent=2)}")

with with_logging_block("Processing Search Results", logger):
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        with with_logging_block(f"Processing Result for Project ID: {project_id or 'Unknown'}", logger):
            if not project_id or not file_cid or not metadata_cid:
                logger.error(f"Missing required data in result: {result_dict}")
                continue

            logger.info(f"File CID: {file_cid}")
            logger.info(f"Metadata CID: {metadata_cid}")

            # Fetch project details
            with with_logging_block("Fetching Project Details", logger):
                project_details = get_account_detail(project_id)
                if not project_details:
                    logger.error(f"No project details found for Project ID: {project_id}.")
                    continue
                logger.info(f"Fetched project details for {project_id}: {project_details}")

            # Parse blockchain data
            with with_logging_block("Parsing Blockchain Data", logger):
                try:
                    blockchain_data = json.loads(project_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding project details JSON for {project_id}: {e}")
                    continue

            # Validate file CID
            with with_logging_block("Validating File CID", logger):
                validation_result = fetch_project_details(file_cid, blockchain_data)
                logger.info(f"Validation Result for {project_id}: {validation_result}")
                if not validation_result["is_valid"]:
                    logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")
                    continue

            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")

            # Process project metadata
            if project_metadata_cid:
                with with_logging_block("Processing Project Metadata", logger):
                    project_metadata = download_json_from_ipfs(project_metadata_cid)
                    logger.info(f"Downloaded project metadata: {project_metadata}")

            # Process linked user details
            if linked_user:
                with with_logging_block(f"Processing Linked User: {linked_user}", logger):
                    user_details = get_account_detail(linked_user)
                    try:
                        user_details = json.loads(user_details)
                    except json.JSONDecodeError as e:
                        logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                        continue

                    user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid")
                    if user_json_ld_cid:
                        user_metadata = download_json_from_ipfs(user_json_ld_cid)
                        logger.info(f"Downloaded user metadata: {user_metadata}")
                    else:
                        logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")

            # Process metadata CID
            if metadata_cid:
                with with_logging_block("Processing Metadata CID", logger):
                    file_metadata = download_json_from_ipfs(metadata_cid)
                    file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                    logger.info(f"Downloaded file metadata: {file_metadata_json}")


2025-01-13 23:09:14.531 | INFO     | __main__:with_logging_block:16 - 
2025-01-13 23:09:14.531 | INFO     | __main__:with_logging_block:17 - STARTING BLOCK: Keyword Search
2025-01-13 23:09:14.531 | INFO     | __main__:with_logging_block:18 - ==================================================
2025-01-13 23:09:14.531 | INFO     | __main__:<module>:45 - Total search results found for keyword 'paper': 16
2025-01-13 23:09:14.531 | INFO     | __main__:<module>:48 - Listing all search results:
2025-01-13 23:09:14.531 | INFO     | __main__:<module>:52 - Result 1: Project ID: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-13 23:09:14.531 | INFO     | __main__:<module>:52 - Result 2: Project ID: 37355@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-13 23:09:14.532 | INFO     | __main__:<module>:52 - Result 3: Project ID: 37355@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-13 23:09:14.532 | INFO     | __main__:<module>: